# An Argo float


This tutorial shows how simple it is to construct a Kernel in Parcels that mimics the [vertical movement of Argo floats](https://www.aoml.noaa.gov/phod/argo/images/argo_float_mission.jpg).


In [ ]:
import numpy as np

# Define the new Kernels that mimic Argo vertical movement
driftdepth = 1000  # maximum depth in m
maxdepth = 2000  # maximum depth in m
vertical_speed = 0.10  # sink and rise speed in m/s
cycletime = (
    10 * 86400
)  # total time of cycle in seconds  # TODO update to "timedelta64[s]"
drifttime = 9 * 86400  # time of deep drift in seconds


def ArgoPhase1(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def SinkingPhase(p):
        """Phase 0: Sinking with vertical_speed until depth is driftdepth"""
        p.ddepth += vertical_speed * dt
        p.cycle_phase = np.where(p.depth + p.ddepth >= driftdepth, 1, p.cycle_phase)
        p.ddepth = np.where(
            p.depth + p.ddepth >= driftdepth, driftdepth - p.depth, p.ddepth
        )

    SinkingPhase(particles[particles.cycle_phase == 0])


def ArgoPhase2(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def DriftingPhase(p):
        """Phase 1: Drifting at depth for drifttime seconds"""
        p.drift_age += dt
        p.cycle_phase = np.where(p.drift_age >= drifttime, 2, p.cycle_phase)
        p.drift_age = np.where(p.drift_age >= drifttime, 0, p.drift_age)

    DriftingPhase(particles[particles.cycle_phase == 1])


def ArgoPhase3(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def SecondSinkingPhase(p):
        """Phase 2: Sinking further to maxdepth"""
        p.ddepth += vertical_speed * dt
        p.cycle_phase = np.where(p.depth + p.ddepth >= maxdepth, 3, p.cycle_phase)
        p.ddepth = np.where(
            p.depth + p.ddepth >= maxdepth, maxdepth - p.depth, p.ddepth
        )

    SecondSinkingPhase(particles[particles.cycle_phase == 2])


def ArgoPhase4(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def RisingPhase(p):
        """Phase 3: Rising with vertical_speed until at surface"""
        p.ddepth -= vertical_speed * dt
        # p.temp = fieldset.temp[p.time, p.depth, p.lat, p.lon]  # sample if fieldset has temperature
        p.cycle_phase = np.where(
            p.depth + p.ddepth <= fieldset.mindepth, 4, p.cycle_phase
        )

    RisingPhase(particles[particles.cycle_phase == 3])


def ArgoPhase5(particles, fieldset):
    def TransmittingPhase(p):
        """Phase 4: Transmitting at surface until cycletime is reached"""
        p.cycle_phase = np.where(p.cycle_age >= cycletime, 0, p.cycle_phase)
        p.cycle_age = np.where(p.cycle_age >= cycletime, 0, p.cycle_age)

    TransmittingPhase(particles[particles.cycle_phase == 4])


def ArgoPhase6(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds
    particles.cycle_age += dt  # update cycle_age

And then we can run Parcels with this 'custom kernel'.

Note that below we use the two-dimensional velocity fields of GlobCurrent, as these are provided as example_data with Parcels.

We therefore assume that the horizontal velocities are the same throughout the entire water column. However, the `ArgoVerticalMovement` kernel will work on any `FieldSet`, including from full three-dimensional hydrodynamic data.

If the hydrodynamic data also has a Temperature Field, then uncommenting the lines about temperature will also simulate the sampling of temperature.


In [ ]:
from datetime import timedelta

import xarray as xr

import parcels

# Load the GlobCurrent data in the Agulhas region from the example_data
example_dataset_folder = parcels.download_example_dataset("GlobCurrent_example_data")

ds = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")

ds = ds.rename(
    {
        "eastward_eulerian_current_velocity": "U",
        "northward_eulerian_current_velocity": "V",
    }
)

# TODO check why we need to explicitly set time here
ds["time"] = [
    np.timedelta64(int(t), "D") + np.datetime64("1900-01-01") for t in ds["time"].values
]

# TODO check how we can get good performance without loading full dataset in memory
ds.load()  # load the dataset into memory

xgcm_grid = parcels.xgcm.Grid(
    ds,
    coords={"X": {"left": "lon"}, "Y": {"left": "lat"}, "T": {"center": "time"}},
    periodic=False,
)
grid = parcels.xgrid.XGrid(xgcm_grid)

U = parcels.Field("U", ds["U"], grid, interp_method=parcels.XLinear)
V = parcels.Field("V", ds["V"], grid, interp_method=parcels.XLinear)
U.units = parcels.GeographicPolar()
V.units = parcels.Geographic()
UV = parcels.VectorField("UV", U, V)

fieldset = parcels.FieldSet([U, V, UV])
fieldset.add_constant("mindepth", 0)

# Define a new Particle type including extra Variables
ArgoParticle = parcels.Particle.add_variable(
    [
        parcels.Variable("cycle_phase", dtype=np.int32, initial=0.0),
        parcels.Variable(
            "cycle_age", dtype=np.float32, initial=0.0
        ),  # TODO update to "timedelta64[s]"
        parcels.Variable("drift_age", dtype=np.float32, initial=0.0),
        # # if fieldset has temperature
        # Variable('temp', dtype=np.float32, initial=np.nan),
    ]
)

# Initiate one Argo float in the Agulhas Current
pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=ArgoParticle, lon=[32], lat=[-31], depth=[0]
)

# combine Argo vertical movement kernel with built-in Advection kernel
kernels = [
    ArgoPhase1,
    ArgoPhase2,
    ArgoPhase3,
    ArgoPhase4,
    ArgoPhase5,
    ArgoPhase6,
    parcels.AdvectionRK4,
]

# Create a ParticleFile object to store the output
output_file = parcels.ParticleFile(
    store="argo_float.zarr",
    outputdt=timedelta(minutes=15),
    chunks=(1, 500),  # setting to write in chunks of 500 observations
)

# Now execute the kernels for 30 days, saving data every 30 minutes
pset.execute(
    kernels,
    runtime=timedelta(days=30),
    dt=timedelta(minutes=15),
    output_file=output_file,
)

Now we can plot the trajectory of the Argo float with some simple calls to netCDF4 and matplotlib


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

ds = xr.open_zarr(output_file.store)
x = ds["lon"][:].squeeze()
y = ds["lat"][:].squeeze()
z = ds["z"][:].squeeze()
ds.close()

fig = plt.figure(figsize=(13, 10))
ax = plt.axes(projection="3d")
cb = ax.scatter(x, y, z, c=z, s=20, marker="o")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_zlabel("Depth (m)")
ax.set_zlim(np.max(z), 0)
plt.show()